# Modern Python

If you haven't been doing python for about the last 3 years, or you haven't been keeping up with all the changes
since 3.7, then python may feel like a new language for you.

This notebook will be a gentle introduction to python for engineers familiar with programming in another language. After
the basics are out of the way, an in depth coverage of modern features will be covered.  This guide will also introduce
you to popular python packages, and how to best manage your python project.

- Prerequisites
    - Installing VS Code
    - Installing python devel deps
    - Installing a python version manager
    - Virtualenvs
- The Basics
    - Basic Data types
        - Maps, lists, sets, int, num, str, bytes, file-like 
    - Intermediate data types
        - Iterables, Sequences, Unions, Intersections, Callable
    - Iterator protocol
    - Scoping
        - Closures
        - nonlocal and global
    - Functions
        - Pass by reference (vs Pass by value)
        - The multitude of argument passing options
        - decorators
    - Modules, packages and imports
    - Classes
        - dataclass types (dataclasses, attrs, pydantic, etc)
        - property
        - slots
    - Exceptions
    - Context Managers
- Advanced
    - Generators
    - asyncio
        - event loop and runners
        - Futures, Tasks, and Coroutines
    - Inheritance
        - Simple Inheritance
        - Multiple Inheritance and MRO
        - Difference between `__new__` and `__init__`
    - Type Theory: [PEP-483](https://peps.python.org/pep-0483/)
        - Protocols/Subtyping: [PEP-544](https://peps.python.org/pep-0544/)
        - ParamSpec: [PEP-612](https://peps.python.org/pep-0612/)
        - TypeVar and Generic: [PEP-484](https://peps.python.org/pep-0484/)
        - Variadic Generics: [PEP-646](https://peps.python.org/pep-0646/)
        - Literal and LiteralString: [PEP-675](https://peps.python.org/pep-0675/)
        - New type annotation syntax: [PEP-695](https://peps.python.org/pep-0695/)

In [1]:
# Install a python development environment
# 
# If you are coming from a brand new environment (eg, a new laptop or installing to a docker image), you should use the
# module as a callable with `python -m excursor.core.installer`.  If you already have the following, you can run
# this notebook instead: 
# - python 3.10+ installed
# - pip installed

from excursor.core.installer import Installer, PythonDevel
from excursor.core.process import Run

# Set up python devel deps

pd = PythonDevel()
async def async_gen():
    installation = [
        pd._install_sysdeps,
        pd._install_asdf,
        pd._install_poetry,
        pd._create_venv
    ]
    for step in installation:
        yield await step()

results = [result async for result in async_gen()]
    


Running command: dnf install -y  curl git make gcc patch zlib-devel bzip2 bzip2-devel readline-devel sqlite sqlite-devel openssl-devel tk-devel libffi-devel xz-devel libuuid-devel gdbm-devel libnsl2-devel which
/bin/sh: 1: dnf: not found
127
Deleting old /home/placeoftheway/.asdf
Running command: git clone https://github.com/asdf-vm/asdf.git ~/.asdf --branch v0.12.0
Cloning into '/home/placeoftheway/.asdf'...
Note: switching to '816195d615427b033a7426a4fb4d7fac4cf2d791'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

Running

: 

## Functions and basic types

We will start with functions and basic types.  A function is declared with `def` followed by a name, and then a parenthesis
with 0 or more parameters.

Python functions has several special abilities with functions:

- Can take default values for parameters
- Can specify keyword at the calling of a function
- Can take positional only parameters using `/`
- Can take keyword only parameters using `*`
- Can pass in a list for positional args with the `*` operator
- Can pass in a dictionary for keyword args using the `**` operator

## Simple function

Here is a basic function.  It takes two arguments, one is named `name` and takes a `str` type.  The second parameter
is called `age` and takes an `int` type.  It returns a `str` denoted by `-> str`

Note that in python, types are purely optional and are calculated by a static type checker.  But at runtime, the python
interpreter does not use any of this information.  The annotations are available with reflection though.

However, you should _always_ use types, even if they optional.

In [ ]:
def basic_with_args(name: str, age: int) -> str:
    """Returns name is age"""
    return f"{name} is {age}"

value = basic_with_args("sean", 51)
print(value)

: 

## Calling by keyword

You can also call by keyword as shown here

In [ ]:
value = basic_with_args("john", age=40)
print(value)
# basic_with_args(name="john", 40) # cant use keywordless args after using one earlier

# Note I can reassign value
value = basic_with_args(name="mary", age=32)
print(value)

value = basic_with_args("jane", age="29")  # note that "29" is invalid type,and checker will complain but this will stull work
print(value)

## Default parameters

You can also set default values.  Some care has to be taken with mutable values.  Also, default parameters can only
be declared after all non-default parameters

The code below also shows an example of a Union type: a type that can be one of several choices.  In Algebraic types, 
it is a Sum type (classes are Product types).

This example also shows how to import built in modules to get access to their functionality.

In [ ]:
import os
from pathlib import Path


def writer(content: str | bytes, output: str | Path | None = None):
    """Writes content to output path

    Parameters
    ----------
    content : str | bytes
        the data to write to file
    output : str | Path | None
        path of file to write to (if none, current directory)
    """
    # We defaulted to None, so we check for it
    if output is None:
        output = Path(os.getcwd()) / "testdata"
    
    # Example of pattern matching
    match content:
        case str():      # if content is a str type
            mode = "w+"  # For text
        case bytes():    # if content is a bytes type
            mode = "wb"  # for binary add a 'b'
    with open(output, mode) as f: 
        f.write(content)

writer("just a test")  # No argument passed to `output` parameter
# Check your directory

test_path = Path("testdata2")
writer("another test", output=test_path)
bin_path = "binary_data"
writer(b'01234', bin_path)


In [ ]:
# delete data files
test_path.unlink()
Path("testdata").unlink()
Path("binary_data").unlink()

## Passing lists and dicts

A common (some might say too common) occurrence in python functions is the ability to pass in lists and dicts to
python functions.  By convention, it's often called `*args` and `**kwargs`.

It's best shown with examples

In [ ]:
from datetime import datetime, timedelta
from typing import Tuple


def lots_of_args(
    mapping: dict,
    answers: Tuple[int, int],
    cwd: str | None,
    completed: bool,
    start: datetime,
    end: datetime,
    timeout: timedelta
):
    print(mapping)
    print(answers)
    print(cwd)
    print(completed)
    print(start)
    print(end)
    print(timeout)
    return end + timeout

args = [
    {1: "foo"},
    (100, 200),
    None
]
kwargs = {
    "completed": True,
    "start": datetime.now(),
    "end": datetime.now(),
    "timeout": timedelta(hours=1)
}
lots_of_args(*args, **kwargs)

## Another example

This is a more generic way to write functions.  This is, IMHO poor style, and should only be done with decorators.

In [ ]:
def generic(*args, **kwargs):
    print(f"arg 1 is {args[0]}")
    print(f"{args[1:]}")
    # example of iteration
    for k,v in kwargs.items():
        print(f"{k} = {v}")


generic("john", [1, 2], datetime.now(), runnable=False)

## Typing out *args and **kwargs

It is possible to type annotate *args and **kwargs which is more acceptable, but still not as clear.

## Scopes

This will be a very brief intro, but hopefully even people using python will learn a thing or two here.  The first thing
we will go over is the concept of `scopes`.  Python, unlike most modern languages is not lexically scoped.

What does `lexically scoped` mean?  Lexical means essentially, words, but it has to do with "boundaries" of your code.
In many languages with curly braces, the braces define a scope.  Variables or other symbols introduced in that scope
(eg the curly braces) only last inside the brace section.

Python is not like that.  First off, it doesn't have curly braces.  But it does have indentation, so you might think
that the indentation acts as a kind of scope, just like curly braces do.  This is incorrect.  Python really only has 
Method (or class) scope, and module scope.

The code below illustrates

In [ ]:
module_scoped = 10

def lookup():
    module_scoped = 100

lookup()
print(f"module_scoped is {module_scoped}") # it's still 10

a_list = [1, 2]

def adder(some_list: list[int], val: int):
    some_list.append(val)

def replacer(some_list: list[int]):
    print(f"id of some_list = {id(some_list)}")
    some_list = []
    print(f"id of some_list is now = {id(some_list)}")

adder(a_list, 3)
print(f"a_list is now {a_list}") # this mutates a_list

print(f"id of a_list is {id(a_list)}")
replacer(a_list) # this is like lookup(), it creates a new binding, leaving the original alone
print(a_list)

In [ ]:
# the global and nonlocal are rarely used.  it's kind of a code smell if they are

def globalizer():
    global foo
    foo = "testing"

globalizer()
print("here's foo {foo}")

name = "sean"

def rename(to: str):
    global name
    name = to

rename("john") # because we use global, we reassign it
print(name)

def wrapper(name: str):
    def inner(greet: str):
        #nonlocal name # try uncommenting this
        name = "sean"
        return f"{greet} {name}"
    print(inner("hi"))
    return name

name = "tony"
print(wrapper(name))


## More scoping gotchas

This is a common source of confusion for new pythonistas

In [ ]:
score = 50 # try setting this to 50

def grader(check: int):
    # dont have to create a "default" grade here,  grade is scoped to the entire function
    # however, try changing score to 50, and commenting out the marked lines
    if check > 90: 
        grade = "A"
    elif check > 80:
        grade = "B"
    elif check > 70:
        grade = "C"
    elif check > 60:
        grade = "D"
    #else:              # try commenting this out
    #    grade = "F"    # and this
    return grade


grader(score)

## Closures and Functions

Understanding scopes is necessary to make use of closures and nested functions.  We already showed several examples
of functions